## Imports

In [1]:
import os

from mmaction.datasets import build_dataset, build_dataloader
from mmaction.models import build_model
from mmcv import Config

/home/sadat/miniconda3/envs/scrambmix/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sadat/miniconda3/envs/scrambmix/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Loading batches

In [2]:
cfg = Config.fromfile('./baseline.py')

In [3]:
os.chdir('../../..')

In [4]:
cfg.data.train

{'type': 'RawframeDataset',
 'ann_file': 'data/hmdb51/annotation_train.txt',
 'data_prefix': 'data/hmdb51/rawframes',
 'pipeline': [{'type': 'SampleFrames',
   'clip_len': 32,
   'frame_interval': 2,
   'num_clips': 1},
  {'type': 'RawFrameDecode'},
  {'type': 'Resize', 'scale': (-1, 256)},
  {'type': 'RandomResizedCrop'},
  {'type': 'Resize', 'scale': (224, 224), 'keep_ratio': False},
  {'type': 'Flip', 'flip_ratio': 0.5},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_bgr': False},
  {'type': 'FormatShape', 'input_format': 'NCTHW'},
  {'type': 'Collect', 'keys': ['imgs', 'label'], 'meta_keys': []},
  {'type': 'ToTensor', 'keys': ['imgs', 'label']}]}

In [5]:
train_dataset = build_dataset(cfg=cfg.data.train)
train_loader = build_dataloader(
        train_dataset,
        videos_per_gpu=5,
        workers_per_gpu=4,
        persistent_workers=False,
        num_gpus=1,
        dist=False)

val_dataset = build_dataset(cfg=cfg.data.val)
val_loader = build_dataloader(
        val_dataset,
        videos_per_gpu=1,
        workers_per_gpu=4,
        persistent_workers=False,
        num_gpus=1,
        dist=False)

## Learning Hyperparameters

In [ ]:
import optuna
import torch.nn as nn
import torch.optim as optim
from mmcv import Config
import torch
import os
import logging

# Configure logging
logging.basicConfig(filename='optuna_training_adam.log', 
                    filemode='w', 
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    level=logging.INFO)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Database file path for saving study
db_file = "sqlite:///optuna_study_adam.db"

sampler = optuna.samplers.TPESampler(seed=42)

# Set up study with the option to resume if it already exists
study = optuna.create_study(
    sampler=sampler,
    direction="maximize", 
    study_name="my_study", 
    storage=db_file,
    load_if_exists=True
)

def objective(trial):
    # Hyperparameters to tune
    dropout_ratio = trial.suggest_float("dropout_ratio", 0.3, 0.7)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    max_norm = trial.suggest_int("max_norm", 10, 50)
    
    # Backbone parameters
    cfg.model.backbone.with_pool2 = trial.suggest_categorical("with_pool2", [True, False])
    cfg.model.backbone.bottleneck_mode = trial.suggest_categorical("bottleneck_mode", ["ir", "ip"])
    cfg.model.backbone.norm_eval = trial.suggest_categorical("norm_eval", [True, False])
    cfg.model.backbone.bn_frozen = trial.suggest_categorical("bn_frozen", [True, False])
    
    # Fixed pretrained URL
    cfg.model.backbone.pretrained = 'https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth'

    # Adjust config parameters
    cfg.model.cls_head.dropout_ratio = dropout_ratio
    
    # Initialize model, criterion, optimizer, scheduler
    model = build_model(cfg.model, train_cfg=None, test_cfg=cfg.get('test_cfg')).to(device)
    
    # Use Adam optimizer
    optimizer = optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=0.00001
    )
    # Step-based learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    # Training and validation
    total_epochs = 20
    eval_interval = 5
    
    for epoch in range(total_epochs):
        # Step scheduler only after each epoch
        scheduler.step()
        
        # Training loop
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for i, data in enumerate(train_loader):
            inputs, labels = data['imgs'].to(device), data['label'].to(device)
            
            optimizer.zero_grad()
            results = model(inputs, labels, return_loss=True)
            loss = results['loss_cls']
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            running_loss += loss.item()
            correct += (results['top1_acc'] * inputs.size(0))
            total += inputs.size(0)

        train_accuracy = correct / total
        train_loss = running_loss / len(train_loader)

        logging.info(f"Epoch [{epoch + 1}/{total_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Validation loop (every `eval_interval` epochs)
        if (epoch + 1) % eval_interval == 0:
            model.eval()
            val_running_loss, val_correct, val_total = 0.0, 0, 0
            with torch.no_grad():
                for val_data in val_loader:
                    val_inputs, val_labels = val_data['imgs'].to(device), val_data['label'].to(device)
                    
                    val_results = model(val_inputs, val_labels, return_loss=True)
                    val_loss = val_results['loss_cls']
                    val_running_loss += val_loss.item()
                    val_correct += (val_results['top1_acc'] * val_inputs.size(0))
                    val_total += val_inputs.size(0)

            val_accuracy = val_correct / val_total
            val_loss = val_running_loss / len(val_loader)

            # Report validation accuracy to Optuna
            trial.report(val_accuracy, epoch)

            # Prune unpromising trials
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
    
    return val_accuracy 

# Run Optuna Study
study.optimize(objective, n_trials=50)

logging.info("Best hyperparameters: %s", study.best_params)
logging.info("Best validation accuracy: %f", study.best_value)

[I 2024-11-06 23:41:27,519] Using an existing study with name 'my_study' instead of creating a new one.
/tmp/ipykernel_2026716/2058400291.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
2024-11-06 23:41:27,945 - mmaction - INFO - load model from: https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth
2024-11-06 23:41:27,946 - mmaction - INFO - load checkpoint from http path: https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth
[I 2024-11-07 00:05:06,926] Trial 40 pruned. 
2024-11-07 00:05:07,312 - mmaction - INFO - load model from: https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth
2024-11-07 00:05:07,313 - mmaction 

KeyboardInterrupt: 

In [7]:
# Retrieve all trials and print their parameters
for trial in study.trials:
    print(f"Trial number: {trial.number}")
    print(f"Parameters: {trial.params}")
    print(f"Value (e.g., validation accuracy): {trial.value}")
    print("-" * 30)

Trial number: 0
Parameters: {'dropout_ratio': 0.4810651569645834, 'lr': 0.0004258940663125744, 'warmup_ratio': 0.1917330003925753, 'max_norm': 50, 'with_pool2': False, 'bottleneck_mode': 'ir', 'norm_eval': False, 'bn_frozen': True}
Value (e.g., validation accuracy): 0.7588235294117647
------------------------------
Trial number: 1
Parameters: {'dropout_ratio': 0.5627200224207228, 'lr': 0.000567154514040872, 'warmup_ratio': 0.16183019246349312, 'max_norm': 42, 'with_pool2': False, 'bottleneck_mode': 'ir', 'norm_eval': True, 'bn_frozen': False}
Value (e.g., validation accuracy): None
------------------------------
Trial number: 2
Parameters: {'dropout_ratio': 0.37629465362426134, 'lr': 7.80756602206136e-05, 'warmup_ratio': 0.11381606982740221, 'max_norm': 39, 'with_pool2': True, 'bottleneck_mode': 'ir', 'norm_eval': True, 'bn_frozen': True}
Value (e.g., validation accuracy): 0.6947712418300653
------------------------------
Trial number: 3
Parameters: {'dropout_ratio': 0.6300026158652796

In [8]:
best_trial = study.best_trial
print("Best trial number:", best_trial.number)
print("Best parameters:", best_trial.params)
print("Best validation accuracy:", best_trial.value)

Best trial number: 20
Best parameters: {'dropout_ratio': 0.526359895507142, 'lr': 0.00041991630239679586, 'warmup_ratio': 0.19984035642104114, 'max_norm': 10, 'with_pool2': False, 'bottleneck_mode': 'ir', 'norm_eval': True, 'bn_frozen': False}
Best validation accuracy: 0.7712418300653594
